In [1]:
import hopsworks
import pandas as pd
import joblib
import os
from sklearn.linear_model import LogisticRegression
import numpy as np

from sklearn.compose import ColumnTransformer
from sklearn.pipeline import Pipeline
from sklearn.impute import SimpleImputer
from sklearn.preprocessing import StandardScaler, OneHotEncoder
from sklearn.feature_selection import SelectPercentile, chi2


In [2]:
key=""
with open("api-key.txt", "r") as f:
    key = f.read().rstrip()
os.environ['HOPSWORKS_PROJECT']="deloitte"
os.environ['HOPSWORKS_HOST']="6a525ee0-91d8-11ed-9cc8-9fe82dc2b6fd.cloud.hopsworks.ai"
os.environ['HOPSWORKS_API_KEY']=key

In [3]:
import hopsworks

proj = hopsworks.login()
fs = proj.get_feature_store()

Connected. Call `.close()` to terminate connection gracefully.

Logged in to project, explore it here https://6a525ee0-91d8-11ed-9cc8-9fe82dc2b6fd.cloud.hopsworks.ai:443/p/119
Connected. Call `.close()` to terminate connection gracefully.


In [4]:
version=1
fv = fs.get_feature_view(name="loans", version=version)

In [5]:
fv.init_batch_scoring(training_dataset_version=1)

In [6]:
def retrieve_data(feature_view):
    batch_data = feature_view.get_batch_data()
    return batch_data

df = retrieve_data(fv)
df

2023-03-10 21:35:13,799 INFO: USE `deloitte_featurestore`
2023-03-10 21:35:14,218 INFO: SELECT `fg0`.`earliest_cr_line_year` `earliest_cr_line_year`, `fg0`.`loan_amnt` `loan_amnt`, `fg0`.`term` `term`, `fg0`.`int_rate` `int_rate`, `fg0`.`installment` `installment`, `fg0`.`sub_grade` `sub_grade`, `fg0`.`home_ownership` `home_ownership`, `fg0`.`annual_inc` `annual_inc`, `fg0`.`verification_status` `verification_status`, `fg0`.`purpose` `purpose`, `fg0`.`dti` `dti`, `fg0`.`open_acc` `open_acc`, `fg0`.`pub_rec` `pub_rec`, `fg0`.`revol_bal` `revol_bal`, `fg0`.`revol_util` `revol_util`, `fg0`.`total_acc` `total_acc`, `fg0`.`initial_list_status` `initial_list_status`, `fg0`.`application_type` `application_type`, `fg0`.`mort_acc` `mort_acc`, `fg0`.`pub_rec_bankruptcies` `pub_rec_bankruptcies`, `fg0`.`zip_code` `zip_code`
FROM `deloitte_featurestore`.`loans_1` `fg0`


,earliest_cr_line_year,loan_amnt,term,int_rate,installment,sub_grade,home_ownership,annual_inc,verification_status,purpose,...,open_acc,pub_rec,revol_bal,revol_util,total_acc,initial_list_status,application_type,mort_acc,pub_rec_bankruptcies,zip_code
0,1944,15000.0,60 months,19.47,393.00,D5,RENT,60000.0,Source Verified,debt_consolidation,...,16.0,1,6166.0,65.6,42.0,w,INDIVIDUAL,6.0,1.0,22690
1,1948,25000.0,36 months,21.18,944.19,E3,MORTGAGE,120000.0,Verified,small_business,...,16.0,0,82469.0,53.5,51.0,f,INDIVIDUAL,16.0,0.0,0
2,1950,10000.0,36 months,10.99,327.34,B2,MORTGAGE,78000.0,Source Verified,credit_card,...,19.0,1,12946.0,53.9,53.0,w,INDIVIDUAL,6.0,0.0,0
3,1950,10000.0,36 months,10.16,323.43,B1,MORTGAGE,62000.0,Verified,credit_card,...,11.0,0,13047.0,45.5,23.0,f,INDIVIDUAL,3.0,0.0,0
4,1950,22500.0,60 months,10.37,310.20,B3,MORTGAGE,45000.0,Verified,home_improvement,...,8.0,0,15011.0,43.9,20.0,f,INDIVIDUAL,1.0,0.0,48052
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
395214,2013,9600.0,36 months,13.67,326.57,C3,MORTGAGE,40000.0,Source Verified,debt_consolidation,...,5.0,0,2454.0,64.6,7.0,w,INDIVIDUAL,2.0,0.0,22690
395215,2013,1500.0,36 months,19.99,55.74,D4,RENT,45000.0,Source Verified,other,...,10.0,0,263.0,1.1,14.0,f,INDIVIDUAL,0.0,0.0,30723
395216,2013,10000.0,36 months,16.99,356.48,D1,RENT,46000.0,Source Verified,debt_consolidation,...,11.0,0,5341.0,27.1,14.0,w,INDIVIDUAL,0.0,0.0,70466
395217,2013,15000.0,36 months,18.25,544.17,D3,MORTGAGE,66000.0,Not Verified,debt_consolidation,...,8.0,0,9750.0,54.2,12.0,w,INDIVIDUAL,1.0,0.0,48052


In [7]:
def get_model(project):
    mr = project.get_model_registry()
    model = mr.get_model("lending_model", version=7)
    model_dir = model.download()
    return joblib.load(model_dir + "/lending_model.pkl")

model = get_model(proj)

Connected. Call `.close()` to terminate connection gracefully.


In [8]:
predictions = model.predict(df)
predictions

array([1, 1, 1, ..., 1, 1, 1])

In [9]:
np.unique(predictions, return_counts=True)

(array([0, 1]), array([ 37405, 357814]))